In [70]:
import numpy as np

class NaiveBayes:
  # all probabilities inside the class are log

  def __init__(self, *, param_func, prob_func, param_priors=None):
    # user-provided function to compute the distribution parameters
    # the function should return an list of parameter arrays (...)
    self.param_func = param_func

    # user-provided function to compute the probabilities from the parameters
    # the input are the parameter arrays and the X
    self.prob_func = prob_func

    self.param_priors = [param_priors] if param_priors is not None else []

  def fit(self, X, y):
    # retrieve the number of samples and features from X
    self.n_samples, self.n_features = X.shape

    # collect the individual classes, and record their counts to compute priors
    self.classes, counts = np.unique(y, return_counts=True)

    # compute the parameters for every class - we transpose the array so that
    # the final dimensions are (parameters, classes, features)
    self.params = np.array(
      [self.param_func(X[c==y], *self.param_priors) for c in self.classes]
    ).transpose(1, 0, 2)

    # compute the priors from the counts
    self.class_probs = np.log(counts/self.n_samples)

    return self

  def posteriors(self, X):
    # reshape X to fit the array dimensions (samples, classes, features)
    X = np.reshape(X, (-1, 1, self.n_features))

    # compute the probabilities of the samples (...)
    probs = np.log(self.prob_func(X, *self.params[:,np.newaxis]))
    return probs.sum(axis=2) + self.class_probs[np.newaxis]

  def predict_proba(self, X):
    exp_post = np.exp(self.posteriors(X))
    return exp_post / exp_post.sum()

  def predict(self, X):
    return self.classes[np.argmax(self.posteriors(X), axis=1)]

In [69]:
gauss(X[0], nb.params[0], nb.params[1])

array([[9.09924795e-01, 4.53409680e-01, 3.82570122e+00, 2.63595072e+00],
       [1.68996887e-01, 3.52695054e-02, 9.34980226e-09, 1.01370778e-07],
       [3.20029198e-02, 1.39707396e-01, 3.69704606e-13, 1.79028790e-10]])

## Gaussian Naive Bayes

In [71]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

X, y = datasets.load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X = StandardScaler().fit_transform(X)
# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

def mean_var(X):
  return [X.mean(axis=0), X.var(axis=0)]
def gauss(x, mean, var):
  return np.exp(-(x-mean)**2/(2*var)) / np.sqrt(var*2*np.pi)

nb = NaiveBayes(param_func=mean_var, prob_func=gauss).fit(X, y)
y_pred = nb.predict(X_test)
(y_pred == y_test).mean()

0.9666666666666667

In [74]:
mu, var = nb.params
print(f'mu:\n{mu}\nsigma:\n{np.sqrt(var)}')

mu:
[[-1.01457897  0.85326268 -1.30498732 -1.25489349]
 [ 0.11228223 -0.66143204  0.28532388  0.1667341 ]
 [ 0.90229674 -0.19183064  1.01966344  1.08815939]]
sigma:
[[0.42281163 0.86382391 0.0977141  0.13732713]
 [0.61914766 0.71509357 0.26440097 0.25768996]
 [0.76273803 0.73491557 0.31053007 0.35789433]]


## Bernoulli Naive Bayes

In [76]:
X_S = [[0,1,1,1,0,0],
    [0,0,1,1,1,0],
    [1,1,0,0,0,0],
    [1,1,0,0,0,1],
    [1,0,1,0,1,0]]
X_E = [[1,1,1,1,1,1,1],
  [0,1,1,1,1,0,0],
  [0,0,1,0,0,1,1],
  [1,0,1,1,1,1,0],
  [1,1,0,0,1,0,0]]

X = np.c_[X_S, X_E].T
y = len(X_S[0])*[0] + len(X_E[0])*[1]

nb = NaiveBayes(
  param_func=lambda X: [X.mean(axis=0)],
  prob_func=lambda x, theta: theta*x + (1-theta)*(1-x)
).fit(X, y)

nb.predict_proba([1,0,1,1,0])

array([[0.19237241, 0.80762759]])

### Bayesian version

In [78]:
# not sure this is MAP actually
def map_bernoulli(X, alpha):
  return [
    (X.sum(axis=0) + alpha[1] - 1) / (len(X) + alpha.sum(axis=0) - 2)
  ]
def bernoulli(x, theta):
  return theta*x + (1-theta)*(1-x)

nb = NaiveBayes(
  param_func=map_bernoulli,
  prob_func=bernoulli,
  param_priors=np.full((2, 5), 2)
).fit(X, y)

nb.predict_proba([1,0,1,1,0])

array([[0.23601128, 0.76398872]])